In [1]:

import subprocess
import os

!source /etc/network_turbo

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value
        
import os
# We need Hugging Face の 镜像网站！
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"


设置成功
注意：仅限于学术用途，不承诺稳定性保证


In [2]:

from transformers import MT5Tokenizer, MT5ForConditionalGeneration
import torch

tokenizer = MT5Tokenizer.from_pretrained('google/mt5-small')
model = MT5ForConditionalGeneration.from_pretrained('google/mt5-small',use_safetensors=True)

# the following 2 hyperparameters are task-specific
max_source_length = 128 # 512
max_target_length = 128

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.mt5.tokenization_mt5.MT5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
# Suppose we have the following 2 training examples:
input_sequence_1 = "Welcome to Beijing"
output_sequence_1 = "欢迎来到北京"

input_sequence_2 = "HuggingFace is a company"
output_sequence_2 = "拥抱脸是一家公司"

# encode the inputs
task_prefix = "translate English to Chinese: "
input_sequences = [input_sequence_1, input_sequence_2]

input_tokens_1 = tokenizer.tokenize(task_prefix + input_sequence_1)
print('input_tokens_1:', input_tokens_1)
output_tokens_1 = tokenizer.tokenize(output_sequence_1)
print('output_tokens_1:', output_tokens_1)

input_tokens_2 = tokenizer.tokenize(task_prefix + input_sequence_2)
print('input_tokens 2:', input_tokens_2)
output_tokens_2 = tokenizer.tokenize(output_sequence_2)
print('output_tokens_2:', output_tokens_2)

encoding = tokenizer(
    [task_prefix + sequence for sequence in input_sequences],
    padding="longest", # pad to the longest sequence in the batch
    max_length=max_source_length,
    truncation=True,
    return_tensors="pt",
)

print('encoding', encoding)
input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

# encode the targets
target_encoding = tokenizer(
    [output_sequence_1, output_sequence_2], padding="longest", max_length=max_target_length, truncation=True
)
labels = target_encoding.input_ids

print('Labels:', labels)


input_tokens_1: ['▁translate', '▁English', '▁to', '▁Chinese', ':', '▁', 'Welcome', '▁to', '▁Beijing']
output_tokens_1: ['▁', '欢迎', '来到', '北京']
input_tokens 2: ['▁translate', '▁English', '▁to', '▁Chinese', ':', '▁Hu', 'gging', 'Face', '▁is', '▁', 'a', '▁company']
output_tokens_2: ['▁', '拥', '抱', '脸', '是一家', '公司']
encoding {'input_ids': tensor([[ 37194,   5413,    288,  17542,    267,    259,  29230,    288,  25463,
              1,      0,      0,      0],
        [ 37194,   5413,    288,  17542,    267,   4691,  30374, 122628,    339,
            259,    262,   5835,      1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Labels: [[259, 72658, 97836, 8467, 1, 0, 0], [259, 117810, 51092, 48211, 229581, 5705, 1]]


In [4]:
# 推理（Inference）
# 使用模型对输入进行翻译预测
model.eval()
with torch.no_grad():
    generated_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=max_target_length,
        num_beams=4,
        early_stopping=True
    )

# 解码生成的输出
predictions = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
print("Predictions:", predictions)

Predictions: ['<extra_id_0>', '<extra_id_0>']


In [5]:

# 确保有pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

max_source_length = 128
max_target_length = 128

# 训练数据
input_sequence_1 = "Welcome to Beijing"
output_sequence_1 = "欢迎来到北京"
input_sequence_2 = "HuggingFace is a company"
output_sequence_2 = "拥抱脸是一家公司"

task_prefix = "translate English to Chinese: "
input_sequences = [input_sequence_1, input_sequence_2]

# 编码输入
encoding = tokenizer(
    [task_prefix + sequence for sequence in input_sequences],
    padding="longest",
    max_length=max_source_length,
    truncation=True,
    return_tensors="pt",
)

input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

# 正确编码目标（用于训练）
target_encoding = tokenizer(
    [output_sequence_1, output_sequence_2], 
    padding="longest", 
    max_length=max_target_length, 
    truncation=True,
    return_tensors="pt"  # 添加这个
)

# 创建标签（将pad_token_id替换为-100）
labels = target_encoding.input_ids.clone()
labels[labels == tokenizer.pad_token_id] = -100

print('Labels shape:', labels.shape)
print('Labels:', labels)

# 改进的推理过程
model.eval()
with torch.no_grad():
    generated_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=max_target_length,
        min_length=3,              # 添加最小长度
        num_beams=4,
        early_stopping=True,
        do_sample=False,           # 确保确定性输出
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

# 解码生成的输出
predictions = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
print("Predictions:", predictions)

Labels shape: torch.Size([2, 7])
Labels: tensor([[   259,  72658,  97836,   8467,      1,   -100,   -100],
        [   259, 117810,  51092,  48211, 229581,   5705,      1]])
Predictions: ['<extra_id_0>.', '<extra_id_0>.']


In [9]:
# 方案3: 快速微调mT5模型
import torch
import torch.nn.functional as F
from torch.optim import AdamW


if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 准备更多训练数据
training_data = [
    ("Welcome to Beijing", "欢迎来到北京"),
    ("HuggingFace is a company", "拥抱脸是一家公司"), 
    ("Hello world", "你好世界"),
    ("How are you", "你好吗"),
    ("Thank you", "谢谢"),
    ("Good morning", "早上好"),
    ("I love you", "我爱你"),
    ("What is your name", "你叫什么名字"),
]

def prepare_batch(data_pairs):
    task_prefix = "translate English to Chinese: "
    inputs = [task_prefix + pair[0] for pair in data_pairs]
    targets = [pair[1] for pair in data_pairs]
    
    # 编码输入
    input_encoding = tokenizer(
        inputs,
        padding="longest",
        max_length=128,
        truncation=True,
        return_tensors="pt"
    )
    
    # 编码目标
    target_encoding = tokenizer(
        targets,
        padding="longest", 
        max_length=128,
        truncation=True,
        return_tensors="pt"
    )
    
    # 创建标签
    labels = target_encoding.input_ids.clone()
    labels[labels == tokenizer.pad_token_id] = -100
    
    return {
        'input_ids': input_encoding.input_ids,
        'attention_mask': input_encoding.attention_mask,
        'labels': labels
    }

# 快速训练
model.train()
optimizer = AdamW(model.parameters(), lr=1e-1)

print("开始快速微调...")
for epoch in range(20):  # 增加epoch数
    batch = prepare_batch(training_data)
    
    optimizer.zero_grad()
    
    outputs = model(
        input_ids=batch['input_ids'],
        attention_mask=batch['attention_mask'],
        labels=batch['labels']
    )
    
    loss = outputs.loss
    print(f"Epoch {epoch+1}/20, Loss: {loss.item():.4f}")
    
    loss.backward()
    optimizer.step()

print("微调完成!")

# 测试微调后的模型
def test_translation(text):
    prompt = f"translate English to Chinese: {text}"
    inputs = tokenizer(prompt, return_tensors="pt")
    
    model.eval()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=128,
            min_length=3,
            num_beams=4,
            early_stopping=True,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id
        )
    
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

# 测试
test_sentences = ["Welcome to Beijing", "HuggingFace is a company", "Hello world"]
for sentence in test_sentences:
    translation = test_translation(sentence)
    print(f"输入: {sentence}")
    print(f"翻译: {translation}")
    print()

开始快速微调...
Epoch 1/20, Loss: 14.7317
Epoch 2/20, Loss: 72.5293
Epoch 3/20, Loss: 15.7920
Epoch 4/20, Loss: 20.7762
Epoch 5/20, Loss: 32.1146
Epoch 6/20, Loss: 18.5944
Epoch 7/20, Loss: 16.9871
Epoch 8/20, Loss: 17.8424
Epoch 9/20, Loss: 15.7748
Epoch 10/20, Loss: 17.7032
Epoch 11/20, Loss: 18.0089
Epoch 12/20, Loss: 15.7482
Epoch 13/20, Loss: 13.5947
Epoch 14/20, Loss: 12.8891
Epoch 15/20, Loss: 12.3074
Epoch 16/20, Loss: 12.6190
Epoch 17/20, Loss: 11.3418
Epoch 18/20, Loss: 8.5241
Epoch 19/20, Loss: 7.5368
Epoch 20/20, Loss: 6.8596
微调完成!
输入: Welcome to Beijing
翻译: 你 你 你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你

输入: HuggingFace is a company
翻译: 你 你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你

输入: Hello world
翻译: 你 你 你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你你